# Imports and Reading Data

In [2]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np_utils
%matplotlib inline
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, concatenate, LeakyReLU
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout, SpatialDropout1D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols
from random import sample

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

import gc
from sklearn.decomposition import PCA

import sys
import os
sys.path.append(os.getcwd())
from importlib import reload
import lib_research
reload(lib_research)
from lib_research import *

import warnings
warnings.filterwarnings('ignore')

reload(model_utils)
from data_science_utils.models import mean_absolute_percentage_error
from data_science_utils.models import median_absolute_percentage_error
from sklearn.metrics import mean_squared_error
import seaborn as sns
from more_itertools import flatten
import dill
from collections import Counter
import operator

from gensim.test.utils import datapath
from gensim.utils import tokenize
from gensim.models.fasttext import load_facebook_model


<module 'lib_research' from '/home/ec2-user/SageMaker/ML_hackathon_2019/lib_research.py'>

<module 'data_science_utils.models' from '/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/data_science_utils/models/__init__.py'>

In [5]:

crawl = datapath(os.getcwd()+"/crawl-300d-2M-subword.bin")
fb_model = load_facebook_model(crawl)

In [28]:
df = pd.read_csv("price_prediction/india-asins.csv",sep='\t',header=None,names=["asin","price","gl","text_raw"])

df_utils.drop_columns_safely(df,["asin","gl"],inplace=True)

In [29]:
df.head()

,price,text_raw
0,157.71,book an easy way to ksr
1,123.18,book lectures on contract 1 general principle...
2,69.00,book bangaru kalalu
3,7660.62,book mbbs first year textbook sets 14 books s...
4,247.04,book e cet 2018 electrical electronics en...


In [30]:
df.shape

(4305099, 2)

In [31]:
df[df.price<100].shape

(217629, 2)

In [32]:
def log_base_n(arr,base):
    return np.log(arr) / np.log(base)

In [33]:
df["log_1"] = np.ceil(log_base_n(df.price+1,1.25)).astype(int)
df["log_2"] = np.ceil(np.log2(df.price+1)).astype(int)
df["log_3"] = np.ceil(np.log10(df.price+1)).astype(int)
df["sqrt"] = np.ceil(np.sqrt(df.price)).astype(int)
df["cbrt"] = np.ceil(np.cbrt(df.price)).astype(int)

In [38]:
df["log_1"] = df["log_1"].astype(str)
df["log_2"] = df["log_2"].astype(str)
df["log_3"] = df["log_3"].astype(str)
df["sqrt"] = df["sqrt"].astype(str)
df["cbrt"] = df["cbrt"].astype(str)

In [41]:
df["log_1"] = "_L@O#G_1_" + df["log_1"]
df["log_2"] = "_L@O#G_2_" + df["log_2"]
df["log_3"] = "_L@O#G_3_" + df["log_3"]
df["sqrt"] = "_SQ@#RT_" + df["sqrt"]
df["cbrt"] = "_CB@#RT_" + df["cbrt"]

In [51]:
df.price = df.price/40
df.price = df.price.astype(int).astype(str)
df.price = "_PR#I$CE_" + df.price

In [55]:
df.head()

,price,text_raw,log_1,log_2,log_3,sqrt,cbrt
0,_PR#I$CE_3,book an easy way to ksr,_L@O#G_1_23,_L@O#G_2_8,_L@O#G_3_3,_SQ@#RT_13,_CB@#RT_6
1,_PR#I$CE_3,book lectures on contract 1 general principle...,_L@O#G_1_22,_L@O#G_2_7,_L@O#G_3_3,_SQ@#RT_12,_CB@#RT_5
2,_PR#I$CE_1,book bangaru kalalu,_L@O#G_1_20,_L@O#G_2_7,_L@O#G_3_2,_SQ@#RT_9,_CB@#RT_5
3,_PR#I$CE_191,book mbbs first year textbook sets 14 books s...,_L@O#G_1_41,_L@O#G_2_13,_L@O#G_3_4,_SQ@#RT_88,_CB@#RT_20
4,_PR#I$CE_6,book e cet 2018 electrical electronics en...,_L@O#G_1_25,_L@O#G_2_8,_L@O#G_3_3,_SQ@#RT_16,_CB@#RT_7


In [67]:
df.dropna(inplace=True)

In [71]:
def sp_join(row):
    s = row["text_raw"]
    fp, sp = s[:int(len(s)/2)], s[int(len(s)/2):]
    return fp + " " +row["log_2"] +" "+ row["cbrt"]+" " + sp 

df["text"] = df.apply(sp_join,axis=1)

In [72]:
df['text'] = df.price+ " "  + df.log_1+ " " + df.text + " " + df.log_3 + " " + df.sqrt

In [73]:
df.head()

,price,text_raw,log_1,log_2,log_3,sqrt,cbrt,text
0,_PR#I$CE_3,book an easy way to ksr,_L@O#G_1_23,_L@O#G_2_8,_L@O#G_3_3,_SQ@#RT_13,_CB@#RT_6,_PR#I$CE_3 _L@O#G_1_23 book an easy w _L@O#G_2...
1,_PR#I$CE_3,book lectures on contract 1 general principle...,_L@O#G_1_22,_L@O#G_2_7,_L@O#G_3_3,_SQ@#RT_12,_CB@#RT_5,_PR#I$CE_3 _L@O#G_1_22 book lectures on contra...
2,_PR#I$CE_1,book bangaru kalalu,_L@O#G_1_20,_L@O#G_2_7,_L@O#G_3_2,_SQ@#RT_9,_CB@#RT_5,_PR#I$CE_1 _L@O#G_1_20 book bangaru _L@O#G_2_7...
3,_PR#I$CE_191,book mbbs first year textbook sets 14 books s...,_L@O#G_1_41,_L@O#G_2_13,_L@O#G_3_4,_SQ@#RT_88,_CB@#RT_20,_PR#I$CE_191 _L@O#G_1_41 book mbbs first year ...
4,_PR#I$CE_6,book e cet 2018 electrical electronics en...,_L@O#G_1_25,_L@O#G_2_8,_L@O#G_3_3,_SQ@#RT_16,_CB@#RT_7,_PR#I$CE_6 _L@O#G_1_25 book e cet 2018 elec...


In [74]:
df[['text']].to_csv("price_prediction/fasttext-training.txt",index=False,header=False,sep='\t',)

In [84]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

# FastText

In [91]:
cos_sim(fb_model.wv["iphone"],fb_model.wv["android"])
fb_model.wv.most_similar(positive=['iphone'], negative=['android'])
fb_model.wv.most_similar(positive=['oneplus'], negative=['android'])


0.7245769

[('iPhone4', 0.368141770362854),
 ('3gs', 0.3644121289253235),
 ('iPone', 0.36181482672691345),
 ('iTouch', 0.3538857698440552),
 ('iphone4', 0.34788134694099426),
 ('3GS', 0.34028321504592896),
 ('Lifeproof', 0.32989436388015747),
 ('3Gs', 0.32855722308158875),
 ('6plus', 0.3278677761554718),
 ('Iphone4', 0.3245745897293091)]

[('Oneplus', 0.3718072474002838),
 ('2Ts', 0.31908345222473145),
 ('KL0', 0.30332058668136597),
 ('XZs', 0.28037065267562866),
 ('HT.', 0.2756316661834717),
 ('4Ts', 0.2743503153324127),
 ('47T', 0.2731747627258301),
 ('ncix', 0.2727956175804138),
 ('6q', 0.26711007952690125),
 ('71V', 0.26589474081993103)]

In [75]:
fb_model.build_vocab(corpus_file="price_prediction/fasttext-training.txt", update=True)

In [95]:
fb_model.train(corpus_file="price_prediction/fasttext-training.txt", 
               total_examples=4305090, epochs=5, total_words=fb_model.corpus_total_words)

In [ ]:
# total_examples = model4.corpus_count

In [ ]:

from gensim.test.utils import get_tmpfile

fname = get_tmpfile("fasttext.model")

fb_model.save(fname)


In [97]:
from gensim.models import FastText
model = FastText.load(fname)

In [99]:
!ls -ltrh | grep "fasttext.model"

In [102]:
os.getcwd()

'/home/ec2-user/SageMaker/ML_hackathon_2019'

In [103]:
!ls /home/ec2-user

anaconda3    nltk_data		    SageMaker			 src
examples     Nvidia_Cloud_EULA.pdf  sample-notebooks		 tools
gensim-data  README		    sample-notebooks-1561939239  tutorials
